**Install Requirements**

In [1]:
import os
import logging

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Subset, DataLoader
from torch.backends import cudnn

import torchvision
from torchvision import transforms
from pacs_dataset import Pacs
from dann import alexdann
from dann import train_src, test_target

from PIL import Image
from tqdm import tqdm

import matplotlib.pyplot as plt
%matplotlib inline

**Set Arguments**

In [2]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'

NUM_CLASSES = 7

BATCH_SIZE = 128     # Higher batch sizes allows for larger learning rates. An empirical heuristic suggests that, when changing
                     # the batch size, learning rate should change by the same factor to have comparable results

LR = 3e-5          # The initial Learning Rate
MOMENTUM = 0.9       # Hyperparameter for SGD, keep this at 0.9 when using SGD
WEIGHT_DECAY = 5e-5  # Regularization, you can keep this at the default

NUM_EPOCHS = 15      # Total number of training epochs (iterations over dataset)
STEP_SIZE = 15       # How many epochs before decreasing learning rate (if using a step-down policy)
GAMMA = 0.1          # Multiplicative factor for learning rate step-down

LOG_FREQUENCY = 10

BASE_FILE_PATH = "DA_RUN18_LR3e-5_ADAMW_EP15_SS15_G01_ALL_TRANSF"

**Define Data Preprocessing**

In [3]:
transforms = transforms.Compose([transforms.Resize(256),      # Resizes short size of the PIL image to 256
                                      transforms.CenterCrop(224),  # Crops a central square patch of the image
                                                                   # 224 because torchvision's AlexNet needs a 224x224 input!
                                                                   # Remember this when applying different transformations, otherwise you get an error
                                      transforms.ToTensor(), # Turn PIL Image to torch.Tensor
                                      transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)) # Normalizes tensor with mean and standard deviation
])

**Prepare Dataset**

In [4]:
ROOT = 'PACS'

source_data = Pacs(ROOT, transform=transforms, source='photo')
target_data = Pacs(ROOT, transform=transforms, source='art_painting')

_, source_labels = source_data.get_img_with_labels()
_, target_labels = target_data.get_img_with_labels()

print(f"# classes source_data: {len(set(source_labels))}")
print(f"# classes val_set: {len(set(target_labels))}")
print(f"source_data: {len(source_data)} elements")
print(f"target_data: {len(target_data)} elements")

# classes source_data: 7
# classes val_set: 7
source_data: 1670 elements
target_data: 2048 elements


**Prepare Dataloaders**

In [5]:
source_dataloader = DataLoader(source_data, batch_size=BATCH_SIZE, shuffle=True, num_workers=4, drop_last=True)
target_dataloader = DataLoader(target_data, batch_size=BATCH_SIZE, shuffle=False, num_workers=4, drop_last=True)

**Prepare Network**

In [1]:
def init_cnn_objects(model):
  
  # Define loss function
  criterion = nn.CrossEntropyLoss() # for classification, we use Cross Entropy
  parameters_to_optimize = model.parameters() # In this case we optimize over all the parameters of AlexNet
  
  #optimizer = optim.SGD(parameters_to_optimize, lr=LR, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
  #optimizer = optim.Adam(parameters_to_optimize, lr=LR,amsgrad=True)
  optimizer = optim.AdamW(parameters_to_optimize, lr=LR,amsgrad=True, weight_decay=WEIGHT_DECAY)
  scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

  return criterion, optimizer, scheduler

dann = alexdann(pretrained=True)
criterion, optimizer, scheduler = init_cnn_objects(dann)
print("******* NET CREATED *******")

NameError: name 'alexdann' is not defined

**Training**

In [ ]:
def simple_train_test(model, source_dataloader, target_dataloader, file_path=BASE_FILE_PATH):
    train_losses = []
    loss_min = -1
    
    model = model.to(DEVICE)
    cudnn.benchmark
    
    current_step = 0
    for epoch in range(NUM_EPOCHS):
        print('Starting epoch {}/{}, LR = {}'.format(epoch+1, NUM_EPOCHS, scheduler.get_lr()))
        cumulative_loss = train_src(model, source_dataloader, optimizer, criterion, scheduler, current_step, DEVICE)
        curr_loss = cumulative_loss / len(source_dataloader)
        train_losses.append(curr_loss)
        if loss_min == -1 or loss_min > curr_loss:
            loss_min = curr_loss
            torch.save(model, f"{file_path}_best_model.pth")
        scheduler.step()
        
    
    accuracy = test_target(model, target_dataloader, criterion, DEVICE) / float(len(target_data))
    print(f"Accuracy on test set: {accuracy}%")